In [1]:
!pip3 install redis
!pip3 install pymysql
!pip3 install cryptography

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import redis   # 导入redis 模块

r = redis.Redis(host='localhost', port=6379, decode_responses=True)  
r.set('name', 'runoob')  # 设置 name 对应的值
print(r['name'])
print(r.get('name'))  # 取出键 name 对应的值
print(type(r.get('name')))  # 查看类型

runoob
runoob
<class 'str'>


In [3]:
# msyql dataset 数据库 数据表说明
# CREATE TABLE IF NOT EXISTS `browser`(
#     `id` INT UNSIGNED AUTO_INCREMENT,
#    `user_id` INT,
#    `post_id` INT NOT NULL,
#    PRIMARY KEY ( `id` )
# )ENGINE=InnoDB DEFAULT CHARSET=utf8;

# CREATE TABLE IF NOT EXISTS `user_like`(
#     `id` INT UNSIGNED AUTO_INCREMENT,
#    `fans_id` INT,
#    `idol_id` INT NOT NULL,
#    PRIMARY KEY ( `id` )
# )ENGINE=InnoDB DEFAULT CHARSET=utf8;

# 请不要修改mysql 数据库中的数据，因为这个是所有人共用的

In [4]:
import pymysql
import random

#连接数据库
conn=pymysql.connect(host = '162.105.146.38' # 连接名称，默认127.0.0.1
,user = 'root' # 用户名
,passwd='123456' # 密码
,port= 23306
,db='dataset' # 数据库名称
,charset='utf8' # 字符编码
)
cur = conn.cursor() # 生成游标对象

sql="select max(fans_id), max(idol_id) from user_like" # SQL语句
cur.execute(sql) # 执行SQL语句
data = cur.fetchall() # 通过fetchall方法获得数据
for i in data[:2]: # 打印输出前2条数据
    print(i)
    
sql="select max(post_id) from browser" # SQL语句
cur.execute(sql) # 执行SQL语句
data = cur.fetchall() # 通过fetchall方法获得数据
# 打印最大值
print(data[0])

(9999, 9999)
(9999,)


In [5]:
#使用set保存每个用户点赞过的帖子集合，和每个帖子点赞过的用户集合
#每个用户点赞过的帖子 key: user_id_ulike
#每个帖子点赞过的用户 key: post_id_plike

#使用string进行帖子点赞量的保存
#每个帖子点赞总数 key: post_id_slike

#使用set保存每个人关注与被关注集合
#关注集 key: user_id_idols
#被关注集 key: user_id_fans

#key通过后缀区分
def get_redis_key(user_id, attr):
    return str(user_id) + str(attr)

In [6]:
#Construct redis
BATCH_SIZE = 10000
MAX_TIMES = 100

rpipe = r.pipeline()

for epoch in range(MAX_TIMES):
    print(epoch)
    sql = "select * from browser LIMIT {}, {}".format(epoch * BATCH_SIZE, BATCH_SIZE)
    cur.execute(sql)
    data = cur.fetchall()
    for i, user_id, post_id in data:
        rpipe.sadd(get_redis_key(user_id, "ulike"), post_id)
        rpipe.sadd(get_redis_key(post_id, "plike"), user_id)
        #使用string进行点赞数的记录，后续单独排序
        rpipe.set(get_redis_key(post_id, "slike"), r.scard(get_redis_key(post_id, "plike")))
   
    rpipe.execute()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
#用户浏览特定帖子时，返回帖子的点赞数
print(r.get("2023slike"))

103


In [8]:
#用户是否点赞过这个帖子，是则输出“yes！”，否则输出“no！”
#该函数表示user_id的用户是否点赞过post_id的帖子
def liked_already(user_id,post_id):
    sis = r.sismember(get_redis_key(post_id, "plike"), user_id)
    if sis == 1:
        print("yes!")
    else:
        print("no!")
        
print(r.smembers('128plike'))
liked_already(131,128)
liked_already(200,128)
liked_already(9954,128)

{'4394', '5432', '800', '199', '8378', '9581', '4270', '904', '4678', '3815', '9954', '9997', '5824', '4533', '4736', '2034', '4042', '4282', '131', '331', '1261', '1550', '2464', '1908', '5685', '6206', '293', '5016', '2448', '1274', '7721', '7803', '8952', '5531', '3764', '1090', '2556', '7151', '753', '4556', '8465', '3822', '1597', '8438', '9194', '8987', '7860', '5057', '3633', '4487', '441', '5815', '7292', '8534', '2371', '9155', '5097', '8110', '5799', '2905', '5354', '8599', '6614', '9144', '7381', '8253', '4607', '6994', '9500', '513', '3641', '8433', '5434', '8633', '9528', '1808', '9454', '2756', '8302', '3455', '4613', '8298', '3304', '8316', '3965', '4627', '9849', '3111', '4327', '4067', '3821', '6825', '2427', '1994', '3139', '3281', '5435', '3985', '4606'}
yes!
no!
yes!


In [9]:
#给出点赞数排名前十的帖子
def get_likes_top():
    likes_ls = []
    post_id_ls = []
    MAX_POST_ID = 9999
    for i in range(1, MAX_POST_ID + 1):
        post_id_ls.append(i)
        likes_ls.append(r.scard(get_redis_key(i, 'plike')))
        
    #翻转排序取最高10位
    likes_new = sorted(zip(likes_ls, post_id_ls), reverse=True)
    return likes_new[0:10]

#打印点赞前十的post_id
tmp = get_likes_top()
for i in range(0, len(tmp)):
    print(tmp[i][1])


7704
2268
6572
5710
3785
3155
893
9905
7604
6


In [10]:
#Construct redis
BATCH_SIZE = 10000
MAX_TIMES = 100
rpipe = r.pipeline()
for epoch in range(MAX_TIMES):
    print(epoch)
    sql = "select * from user_like LIMIT {}, {}".format(epoch * BATCH_SIZE, BATCH_SIZE)
    cur.execute(sql)
    data = cur.fetchall()
    for i, fans_id, idol_id in data:
        rpipe.sadd(get_redis_key(fans_id, "idols"), idol_id)
        rpipe.sadd(get_redis_key(idol_id, "fans"), fans_id)
    rpipe.execute()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
print(r.smembers('3212idols'))
print(r.smembers('3212fans'))
def follow_each_other_set(user_id):
    st = r.sinter(get_redis_key(user_id, "idols"), get_redis_key(user_id, "fans"))
    print(st)
follow_each_other_set(591)

{'5382', '1319', '3791', '7076', '8679', '6122', '2971', '1361', '3151', '748', '7857', '7951', '72', '44', '369', '2922', '1655', '346', '4599', '2240', '7268', '8957', '1391', '621', '3186', '2136', '6714', '3222', '7297', '9085', '1977', '5717', '6233', '8989', '6186', '3493', '2350', '2067', '1610', '2822', '8423', '6638', '1084', '3735', '4399', '3168', '4220', '19', '5877', '679', '5648', '950', '3851', '5863', '2478', '6652', '9053', '2675', '4036', '1167', '3259', '5109', '8809', '9888', '6241', '6926', '4836', '6721', '7583', '812', '8874', '668', '1638', '8163', '3871', '8464', '6016', '2729', '7241', '1779', '1609', '1820', '1192', '9032', '8500', '478', '9878', '6071', '2181', '3549', '201', '577', '7221', '1038', '8316', '2391', '5215', '9405', '9656', '9326', '6690', '8712', '6577', '4731', '3910', '6097', '8651', '2050', '278', '4605'}
{'4394', '4413', '3351', '8318', '2172', '9265', '9390', '8155', '7417', '7259', '3746', '6159', '8357', '8491', '1503', '4848', '5805', 

In [12]:
def get_common_idols_topk(user_id, topk):
    rpipe = r.pipeline()
    common_idols_ls = []
    user_id_ls = []
    MAX_USER_ID = 9999
    for i in range(1,MAX_USER_ID + 1):
        if i == user_id:
            continue
        user_id_ls.append(i)
        rpipe.sinter(get_redis_key(user_id, 'idols'), get_redis_key(i, 'idols'))
        if i % BATCH_SIZE == 0 or i == MAX_USER_ID:
            data = rpipe.execute()
            common_idols_ls += list(zip(map(len, data), user_id_ls))
    return list(zip(*sorted(common_idols_ls)))[1][-topk:]
    #
ls = get_common_idols_topk(1, 3)
print(ls)

(9457, 9517, 9793)


In [13]:
cur.close() # 关闭游标
conn.close() # 关闭连接